In [1]:
import numpy as np
np.random.seed(100)
from keras import Model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Input
import spacy
import numpy as np
from nltk.corpus import movie_reviews
from random import shuffle
import nltk
!python -m spacy download en_core_web_lg
!pip install sentence-transformers
import en_core_web_lg
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
nltk.download('movie_reviews')
nltk.download('punkt')

     |████████████████████████████████| 827.9MB 2.6MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=d0be9057fdcdec44a581b898adf9c5f5514b02e5b4aab1f195b7aebca7c37406
  Stored in directory: /tmp/pip-ephem-wheel-cache-mpd0_rfw/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 92kB 3.9MB/s 
     |████████████████████████████████| 2.5MB 7.3MB/s 
     |████████████████████████████████| 1.2MB 20.8MB/s 
     |████████████████████████████████| 3.3MB 41.3MB/s 
     |████████████████████████████████| 901kB 36.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.0.0-cp37-none-any.whl size=126711 sha256=67c4e50be00d2264b124e1363b473ed4a385242d47e3b72b1bffc964de8f9fdc
  Stored in directory: /root/.cache/pip/wheels/38/d2/9

True

In [ ]:
# Load a spacy model for english
en_model = en_core_web_lg.load()

In [23]:
from sentence_transformers import SentenceTransformer
sent_embed_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [24]:
# check the sentence embedding from the transformers
sentence = "I am home."
sent_embedding = sent_embed_model.encode(sentence)
print(sent_embedding.shape)

(384,)


In [ ]:
for token in en_model('I am taking a lecture on sentiment analysis.'):
    print(token, token.pos_, token.lemma_, token.vector.shape)

I PRON -PRON- (300,)
am AUX be (300,)
taking VERB take (300,)
a DET a (300,)
lecture NOUN lecture (300,)
on ADP on (300,)
sentiment NOUN sentiment (300,)
analysis NOUN analysis (300,)
. PUNCT . (300,)


In [4]:
def create_data_label_pairs(n=100):
    """Create data samples and corresponding labels."""
    postive_file_ids = movie_reviews.fileids(categories=["pos"])[: n]
    negative_file_ids = movie_reviews.fileids(categories=["neg"])[: n]
    sentence_lengths_pos, sentence_lengths_neg = list(), list()
    data_samples, labels = list(), list()
    for file_id in postive_file_ids:
        print(file_id)
        text = movie_reviews.raw(file_id)
        sentence_lengths_pos.append(len(nltk.sent_tokenize(text)))
        for sent in nltk.sent_tokenize(text):
            data_samples.append(sent)
            labels.append("pos")
    for file_id in negative_file_ids:
        print(file_id)
        text = movie_reviews.raw(file_id)
        sentence_lengths_neg.append(len(nltk.sent_tokenize(text)))
        for sent in nltk.sent_tokenize(text):
            data_samples.append(sent)
            labels.append("neg")
    print('Avg Length for POS Samples =', np.mean(sentence_lengths_pos))
    print('Avg Length for NEG Samples =', np.mean(sentence_lengths_neg))
    print('Avg Length for All Samples =', np.mean(sentence_lengths_pos + sentence_lengths_neg))
    return data_samples, labels

In [5]:
def shuffle_items(items):
    """Shuffle items in a list."""
    shuffle(items)
    return items

In [6]:
def select_items_based_on_indexes(items, indexes):
    """Select items in a list on specified indexes."""
    return [items[index] for index in indexes]

In [7]:
# create new samples of data
data_samples, labels = create_data_label_pairs(100)
all_indexes = list(range(len(data_samples)))
shuffle_items(all_indexes)
data_samples = select_items_based_on_indexes(data_samples, all_indexes)
labels = select_items_based_on_indexes(labels, all_indexes)
# Split train and test data
split_point = int(0.8 * len(data_samples))
print(split_point)
train_data, train_labels = data_samples[: split_point], labels[: split_point]
test_data, test_labels = data_samples[split_point:], labels[split_point:]
print(len(test_data))

pos/cv000_29590.txt
pos/cv001_18431.txt
pos/cv002_15918.txt
pos/cv003_11664.txt
pos/cv004_11636.txt
pos/cv005_29443.txt
pos/cv006_15448.txt
pos/cv007_4968.txt
pos/cv008_29435.txt
pos/cv009_29592.txt
pos/cv010_29198.txt
pos/cv011_12166.txt
pos/cv012_29576.txt
pos/cv013_10159.txt
pos/cv014_13924.txt
pos/cv015_29439.txt
pos/cv016_4659.txt
pos/cv017_22464.txt
pos/cv018_20137.txt
pos/cv019_14482.txt
pos/cv020_8825.txt
pos/cv021_15838.txt
pos/cv022_12864.txt
pos/cv023_12672.txt
pos/cv024_6778.txt
pos/cv025_3108.txt
pos/cv026_29325.txt
pos/cv027_25219.txt
pos/cv028_26746.txt
pos/cv029_18643.txt
pos/cv030_21593.txt
pos/cv031_18452.txt
pos/cv032_22550.txt
pos/cv033_24444.txt
pos/cv034_29647.txt
pos/cv035_3954.txt
pos/cv036_16831.txt
pos/cv037_18510.txt
pos/cv038_9749.txt
pos/cv039_6170.txt
pos/cv040_8276.txt
pos/cv041_21113.txt
pos/cv042_10982.txt
pos/cv043_15013.txt
pos/cv044_16969.txt
pos/cv045_23923.txt
pos/cv046_10188.txt
pos/cv047_1754.txt
pos/cv048_16828.txt
pos/cv049_20471.txt
pos/cv050_

In [8]:
index_to_label_dict = {0: "neg", 1: "pos"}
label_to_index_dict = {"neg": 0, "pos": 1}

In [11]:
def create_sentence_vectors(sentences, model):
    """Create sentence vector for each sentence using the model."""
    return model.encode(sentences)

In [25]:
# Find the sentence vectors for the training data
train_sentence_vectors = create_sentence_vectors(train_data, sent_embed_model)
print(train_sentence_vectors.shape)

(5569, 384)


In [26]:
train_labels_indexed = np.array([label_to_index_dict[label] for label in train_labels])
train_labels_indexed = train_labels_indexed.reshape((train_labels_indexed.shape[0], -1))
print(train_labels_indexed.shape)

(5569, 1)


In [48]:
# Define a model and train
input_layer = Input(shape=(384), name='input')
hidden_layer = Dense(100, name='hidden')(input_layer)
output_layer = Dense(2, activation='softmax', name='output')(hidden_layer)
trained_model = Model(input_layer, output_layer, name='Sent-Transformer')
print(trained_model.summary())

Model: "Sent-Transformer"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 384)]             0         
_________________________________________________________________
hidden (Dense)               (None, 100)               38500     
_________________________________________________________________
output (Dense)               (None, 2)                 202       
Total params: 38,702
Trainable params: 38,702
Non-trainable params: 0
_________________________________________________________________
None


In [49]:
# Compile the model
trained_model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [50]:
# Train on the data
trained_model.fit(train_sentence_vectors, train_labels_indexed, batch_size=8, epochs=30, validation_split=0.2)

Epoch 1/30
557/557 [==============================] - 2s 2ms/step - loss: 0.7106 - accuracy: 0.5664 - val_loss: 0.6459 - val_accuracy: 0.6490
Epoch 2/30
557/557 [==============================] - 1s 2ms/step - loss: 0.6475 - accuracy: 0.6275 - val_loss: 0.6322 - val_accuracy: 0.6499
Epoch 3/30
557/557 [==============================] - 1s 2ms/step - loss: 0.6286 - accuracy: 0.6431 - val_loss: 0.6383 - val_accuracy: 0.6364
Epoch 4/30
557/557 [==============================] - 1s 2ms/step - loss: 0.6158 - accuracy: 0.6628 - val_loss: 0.6297 - val_accuracy: 0.6526
Epoch 5/30
557/557 [==============================] - 1s 2ms/step - loss: 0.6147 - accuracy: 0.6699 - val_loss: 0.6357 - val_accuracy: 0.6490
Epoch 6/30
557/557 [==============================] - 1s 2ms/step - loss: 0.6083 - accuracy: 0.6657 - val_loss: 0.6467 - val_accuracy: 0.6158
Epoch 7/30
557/557 [==============================] - 1s 2ms/step - loss: 0.6174 - accuracy: 0.6585 - val_loss: 0.6278 - val_accuracy: 0.6580
Epoch 

In [46]:
# predict on test data
test_sentence_vectors = create_sentence_vectors(test_data, sent_embed_model)
predicted_sentiment_vectors = trained_model.predict(test_sentence_vectors)
print(predicted_sentiment_vectors.shape)

(1393, 2)


In [51]:
# Evaluation
predicted_sentiments = list()
for pred in predicted_sentiment_vectors:
  predicted_sentiments.append(index_to_label_dict[np.argmax(pred)])
print(classification_report(test_labels, predicted_sentiments))

              precision    recall  f1-score   support

         neg       0.60      0.64      0.62       697
         pos       0.61      0.57      0.59       696

    accuracy                           0.61      1393
   macro avg       0.61      0.61      0.61      1393
weighted avg       0.61      0.61      0.61      1393

